In [2]:
import multiprocessing as mp
import pandas as pd
from tifffile import imread,imwrite
import torch
from hydra import initialize, compose
from hydra.utils import instantiate
from torch.utils.data import Dataset
from src.data import DataHandler
# from transformers import Trainer, TrainingArguments
import src.utils as utils
from src.model import ResNet, ConvNext, Dummy, ModelTrainer, ViT
from src.data.dataset import CellPaintingDatasetCached

In [3]:
import sys

In [4]:
# !{sys.executable} -m pip install transformers datasets

In [5]:
import os
os.chdir('/Users/maciej.filanowicz/CellPainting/src')
initialize(version_base="1.2", config_path="src/conf", job_name="test_app")
cfg = compose(config_name="config", return_hydra_config=True)
dataset_config = instantiate(cfg.dataset)
train_config = instantiate(cfg.train)

In [6]:
data_handler = DataHandler(dataset_config)
# data_loaders = data_handler.get_data_loaders()

In [10]:
from transformers import DeiTForImageClassification

In [11]:
model = DeiTForImageClassification.from_pretrained("facebook/deit-base-distilled-patch16-224")

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/333M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/deit-base-distilled-patch16-224 were not used when initializing DeiTForImageClassification: ['cls_classifier.weight', 'distillation_classifier.bias', 'cls_classifier.bias', 'distillation_classifier.weight']
- This IS expected if you are initializing DeiTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DeiTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream t

In [7]:
data_handler.cache_dataset()

In [9]:
import pandas as pd
metadata = pd.read_csv("../data/processed/meta_data.csv")
train_dataset = CellPaintingDatasetCached(
    metadata[metadata.folder_name == "train"], dataset_config, dataset_config.transforms
)
test_dataset = CellPaintingDatasetCached(
    metadata[metadata.folder_name == "test"], dataset_config, dataset_config.transforms
)
val_dataset = CellPaintingDatasetCached(
    metadata[metadata.folder_name == "val"], dataset_config, dataset_config.transforms
)

In [7]:
from transformers import default_data_collator
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
data_collator = default_data_collator
def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

ModuleNotFoundError: No module named 'transformers'

In [ ]:
args = TrainingArguments(
    "test",
    do_train=True,
    do_eval=True,
    logging_steps=20,
    save_steps=20,
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=30,
    dataloader_num_workers=24,
    weight_decay=0.01,
    load_best_model_at_end=True,
    prediction_loss_only=True,
    report_to='wandb',
    logging_dir='logs',
)

In [ ]:
model = ViT().to('cuda')

trainer = Trainer(
    model,
    args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)